In [ ]:
import numpy as np
import os
from simple_cysh import * # includes pandas as pd
%matplotlib inline
os.chdir(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management\ACM_analysis')

In [ ]:
# get all assessments with desired fields
assmt_df = get_cysh_df('Assesment__c', ['Id', 'Type__c', 'Date_Administered__c',
                                        'X0_to_300_Scaled_Score__c', 'Student__c'], rename_id=True)

# get assessment type
assmt_types = get_cysh_df('Picklist_Value__c', ['Id', 'Name'])
assmt_types = assmt_types.rename(columns={'Id': 'Type__c', 'Name':'Assessment Type'})
assmt_df = assmt_df.merge(assmt_types, how='left', on='Type__c')
del assmt_df['Type__c']

In [ ]:
# filter for valid assessments
assmt_df = assmt_df.loc[assmt_df['Assessment Type'].str.contains('NWEA')
                        & (assmt_df['X0_to_300_Scaled_Score__c']>=100)]
# pivot: one student per row, assessments as columns
assmt_df['Key'] = assmt_df['Assessment Type'] + '_' + assmt_df['Date_Administered__c']
assmt_df = assmt_df.pivot_table(index=['Student__c', 'Assessment Type'], columns='Date_Administered__c', values='X0_to_300_Scaled_Score__c')
assmt_df.reset_index(inplace=True)
# determine baseline and calculate growth (use spring where both spring and fall scores exist)
assmt_df['Baseline'] = assmt_df[['2017-07-01', '2017-09-20']].bfill(axis=1).iloc[:, 0]
assmt_df['Growth'] = assmt_df['2018-01-09'] - assmt_df['Baseline']
# remove records where no baseline present
assmt_df = assmt_df.loc[~assmt_df['Baseline'].isnull()]

In [ ]:
# get current year grade level for each student
# assumption: all students advanced 1 grade between spring and fall
stu_sec_df = get_cysh_df('Student_Section__c', ['Student__c', 'Student_Grade__c'])
stu_sec_df.drop_duplicates('Student__c', inplace=True)
assmt_df = assmt_df.merge(stu_sec_df, how='left', on='Student__c')
assmt_df['Grade at Assessment'] = assmt_df['Student_Grade__c'].astype(float) - 1
assmt_df.loc[assmt_df['2017-07-01'].isnull(), 'Grade at Assessment'] = assmt_df['Student_Grade__c'].astype(float)
assmt_df['Season at Assessment'] = 'Spring'
assmt_df.loc[assmt_df['2017-07-01'].isnull(), 'Season at Assessment'] = 'Fall'

In [ ]:
# Investigate how prior year spring scores compare to current year fall
# Also if students have equivalent spring and fall scores, it's likely an entry error
#     and need to determine which date the score actually occured on
StoF_assmt_df = assmt_df[~assmt_df['2017-07-01'].isnull() & ~assmt_df['2017-09-20'].isnull()].copy()
StoF_assmt_df.loc[:,'StoF'] = assmt_df['2017-09-20'] - assmt_df['2017-07-01']
StoF_assmt_df['StoF'].hist()
# TODO: determine which schools duplicate scores came from and reach out

In [ ]:
# load in targets reference (produced in 'Generate Student Targets.ipynb')
targets_df = pd.read_csv('assessment_targets.csv')
targets_df.loc[targets_df['Subject']=='Reading', 'Subject'] = 'NWEA - ELA'
targets_df.loc[targets_df['Subject']=='Mathematics', 'Subject'] = 'NWEA - MATH'
targets_df['Key'] = (targets_df['Season'] + '_' 
                     + targets_df['Subject'] + '_' 
                     + targets_df['Grade'].astype(str) + '.0_'
                     + targets_df['Score'].astype(str) + '.0')

assmt_df['Key'] = (assmt_df['Season at Assessment'] + '_' 
                   + assmt_df['Assessment Type'] + '_'
                   + assmt_df['Grade at Assessment'].astype(str) + '_'
                   + assmt_df['Baseline'].astype(str))                             
assmt_df = assmt_df.merge(targets_df[['Key', 'Target Growth', 'Target Growth SD']], how='left', on='Key')
assmt_df['Growth_v_Target'] = assmt_df['Growth'] - assmt_df['Target Growth']

In [ ]:
# filter out missing assessments
assmt_df = assmt_df.loc[~assmt_df['Growth_v_Target'].isnull()]
# encode as 1 if target met, 0 if not
assmt_df.loc[assmt_df.Growth_v_Target<0, 'Hit_Target?'] = 0
assmt_df.loc[assmt_df.Growth_v_Target>=0, 'Hit_Target?'] = 1

In [ ]:
assmt_df[['Student__c', 'Assessment Type', 'Growth_v_Target', 'Hit_Target?']].to_csv('assessment_growth.csv', index=False)